# 📚 Import Libraries

In [1]:
!pip install timm
!pip install pytorch_metric_learning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.0/510.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.4/111.4 kB 2.7 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from glob import glob
from collections import defaultdict
from tqdm import tqdm
import time
import os 
import copy
import gc

# visualization
import cv2
import matplotlib.pyplot as plt

# Sklearn
from sklearn.model_selection import StratifiedKFold, KFold, StratifiedGroupKFold,GroupKFold 

# PyTorch 
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
from pytorch_metric_learning import losses
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2

# Metrics 
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# For colored terminal text
from colorama import Fore, Back, Style
c_  = Fore.GREEN
sr_ = Style.RESET_ALL

import warnings
warnings.filterwarnings('ignore')

In [3]:
import wandb

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("WANDB")
    wandb.login(key=api_key)
    anonymous = None
except:
    anonymous = "must"
    print('To use your W&B account,\nGo to Add-ons -> Secrets and provide your W&B access token. Use the Label name as WANDB. \nGet your W&B access token from here: https://wandb.ai/authorize')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


# ⚙️ Configuration

In [4]:
class CFG:
    seed          = 913
    debug         = False # set debug=False for Full Training
    comment       = "eff b7 more more 0.15 satruration on data"
    n_flods       = 5
    backbone      = "convnext_tiny"
    train_bs      = 1
    valid_bs      = train_bs
    epochs        = 60
    lr            = 0.00005
    scheduler     = 'CosineAnnealingLR'
    min_lr        = 1e-7
    T_max         = int(30000/train_bs*epochs)+50
    T_0           = 25
    warmup_epochs = 1
    wd            = 0.01
    n_accumulate  = 2#max(1, 32//train_bs)
    device        = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    N_slides      = 45
print(CFG.n_accumulate)

2


In [5]:
def set_seed(seed = 42):
    np.random.seed(seed)
    #random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    print('> SEEDING DONE')
    
set_seed(CFG.seed)

> SEEDING DONE


In [6]:
Data_dir = [f"../input/cleanedrescaled384-{i}/train/*" for i in range(1,9)] #+ [f"../input/mayo-train-images-size1024-n16/train_images_{i}/*" for i in range(1,9)]

# ❗ Data

In [7]:
skip = ["d4c955_0","e26a04_0","e72352_0"]#,"d7dc3e_0","f3e9f6_0","f3e9f6_0","dbd64b_0"]

In [8]:
def get_data_info(paths , train = True):
    img_prop = defaultdict(list)
    
    for i, path in tqdm(enumerate(paths), total = len(paths),desc = "making dataframe"):
        img_info =  path.split('/')[-1]
        if len(img_info.split("_")) == 4:
            patient_id , image_num  , centre_id ,slice_num= img_info.split("_")
        else:
            patient_id , image_num   ,slice_num= img_info.split("_")
        #tl_pixel = tl_pixel.split('.')[0]
        #centre_id = centre_id.split('.jpg')[0]
        
        img_prop['image_id'].append(f"{patient_id}_{image_num}")
        img_prop['patient_id'].append(patient_id)
        img_prop['image_num'].append(image_num)
        img_prop['slice_num'].append(slice_num.split('.jpg')[0])

        img_prop['path'].append(path)
        #img_prop['tl_pixel'].append(tl_pixel)
        
        if train:
            label = train_data[train_data["image_id"]==f"{patient_id}_{image_num}"].label.item()
            
            img_prop['label'].append(label)
            
        
        #img_prop['density'].append(extra_info)
    
    image_data = pd.DataFrame(img_prop)

    image_data.sort_values(by='image_id', inplace=True)
    image_data.reset_index(inplace=True, drop=True)
    #image_data['density'] = image_data['density'].astype(np.float16)
    
    return image_data

In [9]:
train_data = pd.read_csv("../input/mayo-clinic-strip-ai/train.csv")

In [10]:

train_images = []
for path in Data_dir:
    train_images.extend(glob(path))
#print(train_images)
df = get_data_info(train_images)
df.head()

making dataframe: 100%|██████████| 30240/30240 [00:16<00:00, 1844.19it/s]


,image_id,patient_id,image_num,slice_num,path,label
0,21b106_0,21b106,0,13,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
1,21b106_0,21b106,0,37,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
2,21b106_0,21b106,0,4,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
3,21b106_0,21b106,0,23,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
4,21b106_0,21b106,0,39,../input/cleanedrescaled384-3/train/21b106_0_1...,CE


In [11]:
df

,image_id,patient_id,image_num,slice_num,path,label
0,21b106_0,21b106,0,13,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
1,21b106_0,21b106,0,37,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
2,21b106_0,21b106,0,4,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
3,21b106_0,21b106,0,23,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
4,21b106_0,21b106,0,39,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
...,...,...,...,...,...,...
30235,ffec5c_1,ffec5c,1,39,../input/cleanedrescaled384-1/train/ffec5c_1_7...,LAA
30236,ffec5c_1,ffec5c,1,3,../input/cleanedrescaled384-1/train/ffec5c_1_7...,LAA
30237,ffec5c_1,ffec5c,1,42,../input/cleanedrescaled384-1/train/ffec5c_1_7...,LAA
30238,ffec5c_1,ffec5c,1,2,../input/cleanedrescaled384-1/train/ffec5c_1_7...,LAA


In [12]:
old_preds = pd.read_csv("../input/classifier-output/tile infos.csv")
old_preds.label = old_preds.label.apply(lambda x : x.split('(')[1][0]).astype(int)
old_preds["pred"]  = (old_preds.CE > old_preds.LAA).astype(int)
old_preds["confidence_correct_CE"] = (old_preds.label - old_preds.CE).apply(lambda x : max(x,0)) # bigger value = worst example 
old_preds["confidence_correct_LAA"] = (old_preds.label.apply(lambda x : 0 if x else 1) - old_preds.LAA).apply(lambda x : max(x,0))
thr = 0.55 # higher less we drop 
old_preds = old_preds[ ~( (old_preds.confidence_correct_CE > thr) + (old_preds.confidence_correct_LAA > thr) ) ].reset_index(drop = True)
df = df[ df.patient_id.isin(old_preds.img_id) ]

In [13]:
keep = df['label'].value_counts().LAA*2
df = df.drop(df[df['label']=="CE"][keep//2:].index).reset_index(drop=True)
df = df.drop(df[df['label']=="LAA"][keep//2:].index).reset_index(drop=True)
df.head()

,image_id,patient_id,image_num,slice_num,path,label
0,21b106_0,21b106,0,13,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
1,21b106_0,21b106,0,37,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
2,21b106_0,21b106,0,4,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
3,21b106_0,21b106,0,23,../input/cleanedrescaled384-3/train/21b106_0_1...,CE
4,21b106_0,21b106,0,39,../input/cleanedrescaled384-3/train/21b106_0_1...,CE


In [14]:
temp = df.groupby(by="image_id").count()

In [15]:
df["image_id"].nunique()

179

In [16]:
temp[temp["patient_id"]<CFG.N_slides]

,patient_id,image_num,slice_num,path,label
image_id,,,,,


# 🔨 Utility

In [17]:
def load_img(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    img = img.astype('uint16') # original is uint16

    return img

def show_img(img, ground_truth="", pred = "", conf = ""):
    plt.imshow(img)
    plt.title(f'true: {"CE" if ground_truth else "LAA"} | predicted: {pred} | conf: {conf}')
    plt.axis('off')
    

In [18]:
skf = StratifiedGroupKFold(n_splits=CFG.n_flods, shuffle=True, random_state=CFG.seed)

for fold, (train_idx, val_idx) in enumerate(skf.split(df, df['label'], groups = df["image_id"])):
    df.loc[val_idx, 'fold'] = fold
display(df.groupby(['fold','label'])['image_id'].count())

fold  label
0.0   CE        600
      LAA      1200
1.0   CE        650
      LAA      1150
2.0   CE        550
      LAA      1250
3.0   CE        500
      LAA      1298
4.0   CE        650
      LAA      1100
Name: image_id, dtype: int64

# ❗ DataLoaders

In [19]:
class StripAiDataset(Dataset):
    def __init__(self, df, N_slides, label=True ,transforms = None):
        self.df = df
        self.transforms = transforms
        self.image_ids = df['image_id'].unique().tolist()
        if label:
            self.labels = df.label.apply(lambda x: 1 if x == "CE" else 0).tolist()
        self.N_slides = N_slides
    def __len__(self):
        return len(self.image_ids)
  
    def __getitem__(self,index):
        img_id = self.image_ids[index]
        tempdf =self.df[self.df["image_id"] == img_id]
        img_paths = tempdf.path.sample(self.N_slides,replace=True).tolist()
        img = []
        for path in img_paths:  
            temp = self.transforms(image=load_img(path))["image"]
            temp = np.transpose(temp, (2, 0, 1))
            img.append(temp)
        #print(np.array(img).shape)
        img = np.stack(img , axis = 0).astype(np.float32)

        label = tempdf[0:1].label.item()
        label = 1 if label == "CE" else 0

        
    
    
        return torch.tensor(img), torch.tensor(label)

In [20]:
data_transforms = {
    "train": A.Compose([ #A.RandomBrightnessContrast(p=0.2),
                        A.Flip(p=0.5),
                        #A.ChannelShuffle(),
                        #A.RandomGridShuffle(), 
                        #A.ColorJitter(),
                       A.Normalize()], p=1.0),
    "valid": A.Compose([A.Normalize()], p=1.0)
}

In [21]:
def prepare_loaders(fold,N_slides, df, debug=False):
    
    train_df = df.query("fold!=@fold").reset_index(drop=True)
    valid_df = df.query("fold==@fold").reset_index(drop=True)
    
    if debug:
        train_df = train_df.head(10*5)
        valid_df = valid_df.head(10*3)
        
    train_dataset = StripAiDataset(train_df, N_slides, transforms=data_transforms['train'])
    valid_dataset = StripAiDataset(valid_df, N_slides , transforms=data_transforms['valid'])

    train_loader = DataLoader(train_dataset, batch_size=CFG.train_bs if not debug else 3, 
                              num_workers=1, shuffle=True, pin_memory=True, drop_last=False)
    valid_loader = DataLoader(valid_dataset, batch_size=CFG.valid_bs if not debug else 3, 
                              num_workers=1, shuffle=False, pin_memory=True)
    
    return train_loader, valid_loader

In [22]:
train_loader, valid_loader = prepare_loaders(fold=0,N_slides = CFG.N_slides , df=df, debug=0)
img ,label = next(iter(train_loader))
print(img.size(),label.size())

torch.Size([1, 45, 3, 384, 384]) torch.Size([1])


In [23]:
class Flatten(nn.Module):
    def __init__(self, dim=1):
        super().__init__()
        self.dim = dim

    def forward(self, x): 
        input_shape = x.shape
        output_shape = [input_shape[i] for i in range(self.dim)] + [-1]
        return x.view(*output_shape)


In [24]:
class StripModel(nn.Module):

    def __init__(self, model_name, num_classes=2, pretrained=True ,num_instances=CFG.N_slides , path=""):
        super().__init__()
        self.num_instances = num_instances
        self.encoder = timm.create_model(model_name, pretrained=pretrained, num_classes=num_classes)

        
        
        feature_dim = self.encoder.get_classifier().in_features
        self.encoder.head.fc = nn.Identity()
        self.feature_dim = feature_dim
        print(feature_dim)
        
        self.head = nn.Sequential(
            nn.Conv3d(self.num_instances ,1,(1,12,12)), nn.ReLU(inplace=True), Flatten(),
            nn.Linear(feature_dim, 256), nn.ReLU(inplace=True), 
            nn.Linear(256, 64), nn.ReLU(inplace=True), 
            nn.Linear(64, num_classes)
        )


    def forward(self, x):
        # x: bs x N x C x W x W
        bs, _, ch, w, h = x.shape
        x = x.view(bs*self.num_instances, ch, w, h) # x: N bs x C x W x W
        x = self.encoder.forward_features(x) # x: N bs x C' x W' x W'

        # Concat and pool
        #bs2, ch2, w2, h2 = x.shape
        #x = x.view(-1, self.num_instances, ch2, w2, h2).permute(0, 2, 1, 3, 4)\
            #.contiguous().view(bs, ch2, self.num_instances*w2, h2) # x: bs x C' x N W'' x W''
        emb = self.head(x)

        return emb,self.encoder.head(x)
    

        
        
        

In [25]:
#    model     = StripModel(CFG.backbone ,pretrained = True , path = "../input/good0rbad/best_epoch-0-01-0.6605431338125837.bin").to(CFG.device)

In [26]:
model     = timm.create_model(CFG.backbone ,pretrained = True )

Downloading: "https://dl.fbaipublicfiles.com/convnext/convnext_tiny_1k_224_ema.pth" to /root/.cache/torch/hub/checkpoints/convnext_tiny_1k_224_ema.pth


# 🔧 Loss Function

In [27]:
def fetch_scheduler(optimizer):
    if CFG.scheduler == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer,T_max=int(len(train_loader)/CFG.train_bs*CFG.epochs)+50, 
                                                   eta_min=CFG.min_lr)
    elif CFG.scheduler == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,T_0=int(len(train_loader)/CFG.train_bs*CFG.epochs)+50, 
                                                             eta_min=CFG.min_lr)
    elif CFG.scheduler == 'ReduceLROnPlateau':
        scheduler = lr_scheduler.ReduceLROnPlateau(optimizer,
                                                   mode='min',
                                                   factor=0.1,
                                                   patience=7,
                                                   threshold=0.0001,
                                                   min_lr=CFG.min_lr,)
    elif CFG.scheduler == 'ExponentialLR':
        scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.85)
    elif CFG.scheduler == 'OneCycleLR':
        scheduler = lr_scheduler.OneCycleLR(optimizer, max_lr=0.0001, epochs=CFG.epochs,
                                                    steps_per_epoch=min(4000, len(train_loader)),
                                                    pct_start=0.3)
    elif CFG.scheduler == None:
        return None
        
    return scheduler

In [28]:
metrics = accuracy_score, f1_score, precision_score, recall_score
BCELoss = nn.CrossEntropyLoss()
def criterion(y_pred, y_true):
    return BCELoss(y_pred, y_true)

In [29]:

CrossEntropyLoss = nn.CrossEntropyLoss()
def metric(y_pred, y_true):
    return CrossEntropyLoss(y_pred, y_true)

In [30]:
T2loss = losses.NTXentLoss(temperature=0.07)

# 🚄 Training Function

In [31]:
def train_epoch(model, dataloader, criterion, optimizer,scheduler):
    #model.train()
   
    epoch_loss = 0.0
    epoch_acc = 0
    phase = "train"
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    epsilon = 1e-7
    data_size = len(dataloader.dataset)
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Train ')

    for idx,item in pbar:
                images = item[0].to(CFG.device).float()
                classes = item[1].to(CFG.device).long()
                optimizer.zero_grad()
                with torch.set_grad_enabled(True):
                    output,task2 = model(images)
                    #print(output)
                    loss = criterion(output, classes) * 4  + T2loss(task2,classes.repeat(task2.shape[0]))
                    
                    _, preds = torch.max(output, 1)
                    
                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()
                    scheduler.step()
                    epoch_loss += loss.item() * len(output)
                    epoch_acc += torch.sum(preds == classes.data)
                    
                    tp = (classes.data * preds).sum().to(torch.float32)
                    tn = ((1 - classes.data) * (1 - preds)).sum().to(torch.float32)
                    fp = ((1 - classes.data) * preds).sum().to(torch.float32)
                    fn = (classes.data * (1 - preds)).sum().to(torch.float32)
                mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
                current_lr = optimizer.param_groups[0]['lr']
                pbar.set_postfix(train_loss=f'{epoch_loss /((idx + 1) * CFG.train_bs) :0.4f}',lr=f'{current_lr:0.5f}',
                    gpu_mem=f'{mem:0.2f} GB')
    
    
    data_size = len(dataloader.dataset)
    epoch_loss = epoch_loss / data_size
    epoch_acc = epoch_acc.double() / data_size
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
            
  
    return [epoch_loss]

In [32]:
def valid_epoch(model, dataloader, criterion, optimizer):
    model.eval()
    epoch_loss = 0.0
    epoch_acc = 0
    epoch_metr = 0.0
    phase = "val"
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    epsilon = 1e-7
    data_size = len(dataloader.dataset)
    soft = nn.Softmax()
    pbar = tqdm(enumerate(dataloader), total=len(dataloader), desc='Valid ')
    for idx,item in pbar:
            images = item[0].to(CFG.device).float()
            classes = item[1].to(CFG.device).long()
            optimizer.zero_grad()
            with torch.set_grad_enabled(False):
                output, _= model(images)
                loss = criterion(output, classes)
                metr = metric(output, classes)
                _, preds = torch.max(output, 1)
                

                epoch_loss += loss.item() * len(output)
                epoch_metr += metr.item() * len(output)
                epoch_acc += torch.sum(preds == classes.data)
                
                
                tp += (classes.data * preds).sum().to(torch.float32)
                tn += ((1 - classes.data) * (1 - preds)).sum().to(torch.float32)
                fp += ((1 - classes.data) * preds).sum().to(torch.float32)
                fn += (classes.data * (1 - preds)).sum().to(torch.float32)
            mem = torch.cuda.memory_reserved() / 1E9 if torch.cuda.is_available() else 0
            pbar.set_postfix(valid_loss=f'{epoch_loss /((idx + 1) * CFG.valid_bs):0.4f}', metric = f'{epoch_metr  /((idx + 1) * CFG.valid_bs):0.4f}',
                        gpu_memory=f'{mem:0.2f} GB')
   
    data_size = len(dataloader.dataset)
    epoch_loss = epoch_loss / data_size
    epoch_metr = epoch_metr / data_size
    epoch_acc = epoch_acc.double() / data_size
    
    precision = tp / (tp + fp + epsilon)
    recall = tp / (tp + fn + epsilon)
    f1 = 2* (precision*recall) / (precision + recall + epsilon)
    print(soft(output),classes)
    
    return epoch_loss, epoch_acc, precision, recall, f1 , epoch_metr

In [33]:
def train_model(model, dataloaders_dict, criterion, optimizer, scheduler, num_epochs):
    best_acc = np.inf
    metrics = "Loss, acc, precision, recall, f1, CrossEntropy".split(', ')
    logs = {}
    for epoch in range(num_epochs):
        
        
        model.train()
        if epoch >=5: # freez model after 2 epochs 
            for param in model.encoder.parameters():
                param.requires_grad = False
            for param in model.head.parameters():
                param.requires_grad = True

        else:
            for param in model.encoder.parameters():
                param.requires_grad = True
            for param in model.head.parameters():
                param.requires_grad = False
        
        
        train_info = train_epoch(model, dataloaders_dict["train"], criterion, optimizer ,scheduler)
        for idx,met in enumerate(train_info):
            logs[f"train {metrics[idx]}"] = met
            
        valid_info = valid_epoch(model, dataloaders_dict["val"], criterion, optimizer)
        for idx,met in enumerate(valid_info):
            logs[f"val {metrics[idx]}"] = met
        
        wandb.log(logs)    
        epoch_acc =  valid_info[-1]

        if epoch_acc < best_acc:
            print(f"Valid Score Improved ({best_acc:0.4f} ---> {epoch_acc:0.4f})")
            PATH = f"best_epoch-{fold}-{epoch:02d}-{epoch_acc}.bin"
            torch.save(model.state_dict(), PATH)
            best_acc = epoch_acc
        else:
            PATH = f"last_epoch-{fold}-{epoch:02d}-{epoch_acc}.bin"
            torch.save(model.state_dict(), PATH)
        #Focal.gamma -= 0.1


In [34]:
run = wandb.init(project='Strip-Ai-MIL', 
                     config={k:v for k, v in dict(vars(CFG)).items() if '__' not in k},
                     anonymous=anonymous,
                     name=f"fold-{fold}|model-{CFG.backbone}",
                     group=CFG.comment,
                    )

wandb: Currently logged in as: othiro. Use `wandb login --relogin` to force relogin


In [35]:
fold = 4
print(f'#'*15)
print(f'### Fold: {fold}')
print(f'#'*15)


train_loader, valid_loader = prepare_loaders(fold=fold,N_slides = CFG.N_slides, df=df, debug=CFG.debug)
dataloaders_dict = {"train": train_loader, "val": valid_loader}
model     = StripModel(CFG.backbone ,pretrained = True ).to(CFG.device)
optimizer = optim.AdamW(model.parameters(), lr=CFG.lr, weight_decay=CFG.wd)
scheduler = fetch_scheduler(optimizer)
train_model(model, dataloaders_dict, criterion,optimizer,scheduler,CFG.epochs)
run.finish() 

###############
### Fold: 4
###############
768


Train :  25%|██▌       | 36/144 [33:37<1:39:16, 55.16s/it, gpu_mem=0.00 GB, lr=0.00005, train_loss=2.9093]